In [2]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../output/datasets/credit_risk_model_ready.csv')

In [3]:
df

,rev_util,age,debt_ratio,monthly_inc,open_credit,real_estate,dependents,dlq_2yrs,is_income_zero,worst_delinquency_status_60-89 Days Late,worst_delinquency_status_90+ Days Late,worst_delinquency_status_No Delinquency,worst_delinquency_status_Special Code Late
0,0.006999,38,0.302150,5440.0,4,1,3,0,0,False,False,True,False
1,0.704592,63,0.471441,8000.0,9,1,0,0,0,False,False,True,False
2,0.063113,57,0.068586,5000.0,17,0,0,0,0,False,False,True,False
3,0.368397,68,0.296273,6250.0,16,2,0,0,0,False,False,True,False
4,1.000000,34,0.000000,3500.0,0,0,1,0,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16709,1.000000,46,2.932614,401.0,3,0,2,1,0,False,True,False,False
16710,1.135552,41,0.845887,7500.0,12,4,0,1,0,True,False,False,False
16711,0.920107,31,0.176732,1125.0,4,0,0,1,0,False,True,False,False
16712,0.983825,55,0.064116,4600.0,2,0,6,1,0,False,True,False,False


In [ ]:
label = 'dlq_2yrs'

# Split data into training+validation (85%) and test (15%) sets
train_val_data, test_data = train_test_split(
    df, 
    test_size=0.15, 
    random_state=42, 
    stratify=df[label]
)

# Split the remaining data into training (70%) and validation (15%) sets
# Note: 0.15 / 0.85 ≈ 0.1765
train_data, val_data = train_test_split(
    train_val_data, 
    test_size=0.1765,
    random_state=42, 
    stratify=train_val_data[label]
)

print(f"Original data shape:   {df.shape}")
print(f"Training data shape:   {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape:       {test_data.shape}")

Original data shape:   (16714, 13)
Training data shape:   (11698, 13)
Validation data shape: (2508, 13)
Test data shape:       (2508, 13)


In [ ]:
# Initialize the TabularPredictor
predictor = TabularPredictor(
    label=label,
    problem_type='binary',
    eval_metric='f1',
    path='../output/models/AutogluonModels/credit_risk_3way',
    log_to_file=True
).fit(
    train_data=train_data,
    tuning_data=val_data,
    use_bag_holdout=True,
    time_limit=3600,
    presets='best_quality'
)